In [ ]:
#default_exp HephaestusModel

In [ ]:
#export
#hide
from typing import Union, List, Optional, Tuple
import os
import subprocess
import re
from copy import deepcopy

import sys
sys.path.append("..")

from hephaestus.EditOperations import *
from hephaestus.CondenseEditOperations import *
from hephaestus.AbstractMethod import *
from hephaestus.IOUtils import *

In [ ]:
#hide
from nbdev.showdoc import *

# HephaestusModel

> Encapsulates NMT operations on AbstractMethods.

In [ ]:
#export
class HephaestusModel:
    """
    The `HephaestusModel` is the means through which buggy AbstractMethods are translated into fixed ones. Each
    `HephaestusModel` occupies a directory which contains stored models, vocabularies, and configuration files.

    Required args:
    - `modelDir`: The directory which stores files pertaining to the model. You can use a directory which already
      contains the necessary files (previously generated from a different `HephaestusModel`), in which case the
      model will not have to be trained again. If you provide a directory that does not exist, the `HephaestsuModel`
      will attempt to create it.
    """

    def __init__(self, modelDir: str) -> None:

        # set up constants
        self.__MODEL_DIR =         modelDir if os.path.sep == "/" else modelDir.replace("/", os.path.sep)
        self.__CONFIG_PATH =       os.path.join(self.__MODEL_DIR, "config.yaml")
        self.__SAVE_DATA_PATH =    os.path.join(self.__MODEL_DIR, "save_data")
        self.__SOURCE_VOCAB_PATH = os.path.join(self.__MODEL_DIR, "save_data.vocab.src")
        self.__TARGET_VOCAB_PATH = os.path.join(self.__MODEL_DIR, "save_data.vocab.tgt")
        self.__RAW_OUTPUT_PATH =   os.path.join(self.__MODEL_DIR, "raw_output.txt")
        self.__POST_OUTPUT_PATH =  os.path.join(self.__MODEL_DIR, "postprocessed_output.txt")
        self.__SAVE_MODEL_PREFIX = "model"
        self.__SAVE_MODEL_PATH =   os.path.join(self.__MODEL_DIR, self.__SAVE_MODEL_PREFIX)
        self.__FINAL_MODEL_PATH =  os.path.join(self.__MODEL_DIR, self.__SAVE_MODEL_PREFIX + "_final.pt")

        # create the modelDir directory if it doesn't already exist
        if not os.path.isdir(self.__MODEL_DIR):
            os.mkdir(self.__MODEL_DIR)

    def train(self,

        trainSource: str,
        trainTarget: str,
        validSource: str,
        validTarget: str,

        vocabSamples: int =       10000,
        numGPUs: int =                1,
        trainSteps: int =          1000,
        validSteps: int =          None,
        saveCheckpointSteps: int = None

    ) -> None:
        """
        Trains the model with the given parameters. Files containing AbstractMethods should have one per line with
        tokens separated by spaces. 'source' files must contain AbstractMethods. 'target' files may contain
        AbstractMethods or CompoundOperations.

        As the training progesses, checkpoint model files are created which follow the format `model_step_#.pt`, where
        `#` corresponds to the training step number. Once training is complete, the finalized model is outputted to
        `model_final.pt`.

        Required args:
        - `trainSource`: File name contatining training source data, must be buggy AbstractMethods.
        - `trainTarget`: File name contatining training target data, can be fixed AbstractMethods or CompoundOperations
          describing buggy -> fixed.
        - `validSource`: File name contatining validation source data, must be buggy AbstractMethods.
        - `validTarget`: File name contatining validation target data, must be the same type of data provided in
          `trainTarget`.

        Optional args:
        - `vocabSamples`: Number of transformed samples per corpus to use when building vocabulary. Defaults to 10000.
        - `numGPUs`: Number of GPUs to use concurrently during training. If set to 0, then the CPU is used. Defaults to
          1.
        - `trainSteps`: Number of training steps to go through. Defaults to 1000.
        - `validSteps`: Number of training steps after which each validation occurs; e.g. if `trainSteps` is 1000 and
          `validSteps` is 500, then validation will occur after training steps 500 and 1000. Defaults to
          `trainSteps` / 2.
        - `saveCheckpointSteps`: Number of training steps after which model checkpoints are saved; e.g. if `trainSteps`
          is 1000 and 'saveCheckpointSteps' is 500, then the model will be saved in after training steps 5000 and 1000.
          Defaults to `trainSteps` / 2.
        """

        # TODO: Right now this is just the default model, we should tune the parameters so that it's
        # somewhat specialized to our use case and it works better.

        # determine number of validation and checkpoint steps if none were given
        if validSteps is None:
            validSteps = max(trainSteps // 2, 1)
        if saveCheckpointSteps is None:
            saveCheckpointSteps = max(trainSteps // 2, 1)

        # write config file
        self.__writeConfigFile(trainSource, trainTarget, validSource, validTarget, numGPUs = numGPUs,
                trainSteps = trainSteps, validSteps = validSteps, saveCheckpointSteps = saveCheckpointSteps)
        
        # build vocabulary
        runCommand('onmt_build_vocab -config "{}" -n_sample {}'.format(self.__CONFIG_PATH, vocabSamples))

        # delete previous model files
        for file in os.listdir(self.__MODEL_DIR):
            if re.search(r"^" + self.__SAVE_MODEL_PREFIX + r"_(?:step_[0-9]+|final).pt$", file):
                os.remove(os.path.join(self.__MODEL_DIR, file))
    
        # train the model
        runCommand('onmt_train -config "{}"'.format(self.__CONFIG_PATH))

        # find and release the highest trained model
        latestModel = None
        maxNum = 0
        for file in os.listdir(self.__MODEL_DIR):
            match = re.search(r"^" + self.__SAVE_MODEL_PREFIX + r"_(?:step_([0-9]+)|final).pt$", file)
            if match:
                stepNum = int(match.group(1))
                if stepNum > maxNum:
                    latestModel = os.path.join(self.__MODEL_DIR, file)
                    maxNum = stepNum

        if latestModel is not None:
            runCommand('onmt_release_model --model "{}" --output "{}"'.format(latestModel, self.__FINAL_MODEL_PATH))

    def translate(self,
        buggy: Union[str, AbstractMethod, List[AbstractMethod]],
        modelFile: str = None,
        applyEditOperations: bool = True
    ) -> Union[Optional[AbstractMethod], List[Optional[AbstractMethod]]]:
        """
        Translates the given `buggy` AbstractMethods into supposedly fixed AbstractMethods, writes them to
        `<model_directory>/postprocessed_output.txt`, and then returns them. The raw output of the model is written
        to `<model_directory>/raw_output.txt` in case you want to access that as well. Depending on what type of
        value is passed to `buggy`, the return value of this method changes according to the following:

        | `buggy` type           | Return type                      |
        | :--------------------- | :------------------------------- |
        | `str` (a file)         | `List[Optional[AbstractMethod]]` |
        | `AbstractMethod`       | `Optional[AbstractMethod]`       |
        | `List[AbstractMethod]` | `List[Optional[AbstractMethod]]` |

        A `None` return value means that the model was unable to translate that abstract method correctly. This
        could be due to the model outputting non well-formed CompoundOperations, among other things. These will
        appear as blank lines in `postprocessed_output.txt`.

        Optional args:
        - `modelFile`: A `.pt` file which is used for translation instead of the default `model_final.pt`
        - `applyEditOperations`: When set to True, the model output is interpreted as CompoundOperations and a
          postprocessing stage occurs where the outputted CompoundOperations are applied to the inputted
          AbstractMethods. When set to False, the raw output is interpreted as AbstractMethods and returned
          without a postprocessing stage; in this case, the contents of `raw_output.txt` and
          `postprocessed_output.txt` are identical. If the model was trained with EditOperations,
          `applyEditOperations` should be True; if the model was trained with just AbstractMethods as in for
          the control group, then this should be False. Defaults to True.
        """

        # determine which model file to use, and raise an error if it doesn't exist
        if modelFile is None:
            modelFile = self.__FINAL_MODEL_PATH
        if not os.path.isfile(modelFile):
            raise FileNotFoundError("Hephaestus: model not found -- {}".format(modelFile))
        
        # write the AbstractMethods to a file if they were given directly
        buggyFile = None
        if type(buggy) in (AbstractMethod, list):
            buggyFile = os.path.join(self.__MODEL_DIR, "input.txt")
            writeAbstractMethodsToFile(buggyFile, buggy if type(buggy) is list else [buggy])
        else:
            buggyFile = buggy
        
        # translate the buggy methods
        command = 'onmt_translate -model "{}" -src "{}" -output "{}"'.format(modelFile, buggyFile, self.__RAW_OUTPUT_PATH)
        if getYamlParameter(self.__CONFIG_PATH, "world_size") is not None: # if GPU should be used
            command += "-gpu 0"
        runCommand(command)

        # strip the last line of the output file, as OpenNMT likes to put a newline at the end
        with open(self.__RAW_OUTPUT_PATH, "r+") as outputFile:
            lines = outputFile.readlines()
            lines[-1] = lines[-1].strip()
            outputFile.seek(0)
            outputFile.writelines(lines)
            outputFile.truncate()

        # get all inputted AbstractMethods
        inputMethods = []
        if type(buggy) in (AbstractMethod, list):
            inputMethods = buggy if type(buggy) is list else [buggy]
        else:
            inputMethods = readAbstractMethodsFromFile(buggyFile)
        
        # If edit ops should be applied, then extract the operations from the output file and attempt to
        # apply them to the input methods. Assign a None value to a fixed method if its corresponding
        # operations were not able to be read. Copy input methods before applying edit operations so that
        # the original remains unmodified.
        fixedMethods = []
        if applyEditOperations:
            operations = readCompoundOperationsFromFile(self.__RAW_OUTPUT_PATH)
            for inputMethod, opList in zip(inputMethods, operations):
                if opList is None:
                    fixedMethods.append(None)
                else:
                    try:
                        inputMethodCopy = deepcopy(inputMethod)
                        inputMethodCopy.applyEditOperations(opList)
                        fixedMethods.append(inputMethodCopy)
                    except:
                        fixedMethods.append(None)
        
        # Simply interpret the output as abstract methods if not interpreting as edit operations
        else:
            fixedMethods = readAbstractMethodsFromFile(self.__RAW_OUTPUT_PATH)
        
        # substitute None for empty AbstractMethods and write the fixed methods to the postprocessed output file
        writeAbstractMethodsToFile(
            self.__POST_OUTPUT_PATH,
            [" " if method is None else method for method in fixedMethods]
        )

        # return fixed methods
        return fixedMethods if type(buggy) is list else fixedMethods[0]

    def __writeConfigFile(self, *args, **kwargs) -> None:
        """
        Creates the config file.
        """
        
        lines = [
            "# AUTOGENERATED",
            "",
            "# Samples will be written to here",
            "save_data: {}".format(self.__SAVE_DATA_PATH.replace(os.path.sep, "/")),
            "",
            "# Vocabs will be written to these files",
            "src_vocab: {}".format(self.__SOURCE_VOCAB_PATH.replace(os.path.sep, "/")),
            "tgt_vocab: {}".format(self.__TARGET_VOCAB_PATH.replace(os.path.sep, "/")),
            "",
            "# Allow overwriting existing files in the directory",
            "overwrite: True",
            "",
            "# Data corpus",
            "data:",
            "    corpus_1:",
            "        path_src: {}".format(args[0].replace(os.path.sep, "/")),
            "        path_tgt: {}".format(args[1].replace(os.path.sep, "/")),
            "        transforms: []",
            "        weight: 1",
            "    valid:",
            "        path_src: {}".format(args[2].replace(os.path.sep, "/")),
            "        path_tgt: {}".format(args[3].replace(os.path.sep, "/")),
            "        transforms: []",
            "",
            "# Checkpoints will be saved here",
            "save_model: {}".format(self.__SAVE_MODEL_PATH.replace(os.path.sep, "/")),
            "save_checkpoint_steps: {}".format(kwargs["saveCheckpointSteps"]),
            "train_steps: {}".format(kwargs["trainSteps"]),
            "valid_steps: {}".format(kwargs["validSteps"]),
            ""
        ]

        numGPUs = kwargs["numGPUs"]
        if numGPUs > 0:
            lines += [
                "# Train using {} GPU{}".format(numGPUs, "s" if numGPUs > 1 else ""),
                "world_size: {}".format(numGPUs),
                "gpu_ranks:",
                *["- {}".format(i) for i in range(numGPUs)]
            ]
        else:
            lines += ["# Train using the CPU, so no world_size provided"]

        with open(self.__CONFIG_PATH, "w") as file:
            file.write("\n".join(lines))

In [ ]:
show_doc(HephaestusModel.train)

<h4 id="HephaestusModel.train" class="doc_header"><code>HephaestusModel.train</code><a href="__main__.py#L32" class="source_link" style="float:right">[source]</a></h4>

> <code>HephaestusModel.train</code>(**`trainSource`**:`str`, **`trainTarget`**:`str`, **`validSource`**:`str`, **`validTarget`**:`str`, **`vocabSamples`**:`int`=*`10000`*, **`numGPUs`**:`int`=*`1`*, **`trainSteps`**:`int`=*`1000`*, **`validSteps`**:`int`=*`None`*, **`saveCheckpointSteps`**:`int`=*`None`*)

Trains the model with the given parameters. Files containing AbstractMethods should have one per line with
tokens separated by spaces. 'source' files must contain AbstractMethods. 'target' files may contain
AbstractMethods or CompoundOperations.

As the training progesses, checkpoint model files are created which follow the format `model_step_#.pt`, where
`#` corresponds to the training step number. Once training is complete, the finalized model is outputted to
`model_final.pt`.

Required args:
- `trainSource`: File name contatining training source data, must be buggy AbstractMethods.
- `trainTarget`: File name contatining training target data, can be fixed AbstractMethods or CompoundOperations
  describing buggy -> fixed.
- `validSource`: File name contatining validation source data, must be buggy AbstractMethods.
- `validTarget`: File name contatining validation target data, must be the same type of data provided in
  `trainTarget`.

Optional args:
- `vocabSamples`: Number of transformed samples per corpus to use when building vocabulary. Defaults to 10000.
- `numGPUs`: Number of GPUs to use concurrently during training. If set to 0, then the CPU is used. Defaults to
  1.
- `trainSteps`: Number of training steps to go through. Defaults to 1000.
- `validSteps`: Number of training steps after which each validation occurs; e.g. if `trainSteps` is 1000 and
  `validSteps` is 500, then validation will occur after training steps 500 and 1000. Defaults to
  `trainSteps` / 2.
- `saveCheckpointSteps`: Number of training steps after which model checkpoints are saved; e.g. if `trainSteps`
  is 1000 and 'saveCheckpointSteps' is 500, then the model will be saved in after training steps 5000 and 1000.
  Defaults to `trainSteps` / 2.

In [ ]:
show_doc(HephaestusModel.translate)

<h4 id="HephaestusModel.translate" class="doc_header"><code>HephaestusModel.translate</code><a href="__main__.py#L114" class="source_link" style="float:right">[source]</a></h4>

> <code>HephaestusModel.translate</code>(**`buggy`**:`Union`\[`str`, [`AbstractMethod`](/hephaestus/AbstractMethod.html), `List`\[[`AbstractMethod`](/hephaestus/AbstractMethod.html)\]\], **`modelFile`**:`str`=*`None`*, **`applyEditOperations`**:`bool`=*`True`*)

Translates the given `buggy` AbstractMethods into supposedly fixed AbstractMethods, writes them to
`<model_directory>/postprocessed_output.txt`, and then returns them. The raw output of the model is written
to `<model_directory/raw_output.txt>` in case you want to access that as well. Depending on what type of
value is passed to `buggy`, the return value of this method changes according to the following:

| `buggy` type           | Return type                      |
| :--------------------- | :------------------------------- |
| `str` (a file)         | `List[Optional[AbstractMethod]]` |
| [`AbstractMethod`](/hephaestus/AbstractMethod.html)       | `Optional[AbstractMethod]`       |
| `List[AbstractMethod]` | `List[Optional[AbstractMethod]]` |

A `None` return value means that the model was unable to translate that abstract method correctly. This
could be due to the model outputting non well-formed CompoundOperations, among other things. These will
appear as blank lines in `postprocessed_output.txt`.

Optional args:
- `modelFile`: A `.pt` file which is used for translation instead of the default `model_final.pt`
- `applyEditOperations`: When set to True, the model output is interpreted as CompoundOperations and a
  postprocessing stage occurs where the outputted CompoundOperations are applied to the inputted
  AbstractMethods. When set to False, the raw output is interpreted as AbstractMethods and returned
  without a postprocessing stage; in this case, the contents of `raw_output.txt` and
  `postprocessed_output.txt` are identical. If the model was trained with EditOperations,
  `applyEditOperations` should be True; if the model was trained with just AbstractMethods as in for
  the control group, then this should be False. Defaults to True.

## Example Usage

Let's create a small test model.

In [ ]:
model = HephaestusModel("test_model_loose")

Now there is a directory called `test_model_loose` which will be populated with files once the model is trained. It's time to train the model with the loosely condensed edit operations dataset. Since this is just an example, a *very* small number of training steps will be used.

In [ ]:
# collapse_output
model.train(
    "../data/abstract_methods/small/train_buggy.txt",
    "../data/edit_ops/loose/small/train.txt",
    "../data/abstract_methods/small/valid_buggy.txt",
    "../data/edit_ops/loose/small/valid.txt",
    numGPUs = 0,
    trainSteps = 500,
    validSteps = 100,
    saveCheckpointSteps = 250
)

[2021-04-19 05:48:46,172 INFO] Counter vocab from 10000 samples.
[2021-04-19 05:48:46,172 INFO] Build vocab on 10000 transformed examples/corpus.
[2021-04-19 05:48:48,403 INFO] Counters src:411
[2021-04-19 05:48:48,403 INFO] Counters tgt:405
[2021-04-19 05:48:50,852 WARNING] You have a CUDA device, should run with -gpu_ranks
[2021-04-19 05:48:50,854 INFO] Parsed 2 corpora from -data.
[2021-04-19 05:48:50,855 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2021-04-19 05:48:50,855 INFO] Loading vocab from text file...
[2021-04-19 05:48:50,855 INFO] Loading src vocabulary from test_model_loose/save_data.vocab.src
[2021-04-19 05:48:50,882 INFO] Loaded src vocab has 411 tokens.
[2021-04-19 05:48:50,883 INFO] Loading tgt vocabulary from test_model_loose/save_data.vocab.tgt
[2021-04-19 05:48:50,902 INFO] Loaded tgt vocab has 405 tokens.
[2021-04-19 05:48:50,902 INFO] Building fields with vocab in counters...
[2021-04-19 05:48:50,904 INFO]  * tgt vocab size: 409.
[2021

Now that the model is trained, we can test it out. This gets the first buggy `AbstractMethod` from the testing data.

In [ ]:
buggyMethod = readAbstractMethodsFromFile("../data/abstract_methods/small/test_buggy.txt")[0]
buggyMethod

private TYPE_1 getType ( TYPE_2 VAR_1 ) { TYPE_3 VAR_2 = new TYPE_3 ( STRING_1 ) ; return new TYPE_1 ( VAR_2 , VAR_2 ) ; }

Then translate the method into a supposedly fixed version using `HephaestusModel.translate`.

In [ ]:
#collapse_output
outputMethod = model.translate(buggyMethod)

[2021-04-19 14:08:08,666 INFO] Translating shard 0.
C:\Users\aidan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\onmt\translate\beam_search.py:275: UserWarning: An output with one or more elements was resized since it had shape [5], which does not match the required output shape [1, 5].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  ..\aten\src\ATen\native\Resize.cpp:19.)
  torch.mul(self.topk_scores, length_penalty, out=self.topk_log_probs)
[2021-04-19 14:08:08,776 INFO] PRED AVG SCORE: -0.6497, PRED PPL: 1.9150


In [ ]:
outputMethod

public TYPE_1 getType ( TYPE_2 VAR_1 ) { TYPE_3 VAR_2 = new TYPE_3 ( STRING_1 ) ; return new TYPE_1 ( VAR_2 , VAR_2 ) ; }

Hmmm, the `buggyMethod` and `outputMethod` look very similar. What exactly changed from the former to the latter?

In [ ]:
operations = buggyMethod.getEditOperationsTo(outputMethod)
operations

[REPLACE 0 -> 'public']

So there was only one operation done on the inputted buggy method, which was to replace the token at index 0 with `"public"`, i.e. `"private"` was changed to `"public"`. We can verify that these were the actual edit operations applied by the model by looking at `raw_output.txt` directly.

In [ ]:
compoundOps = readCompoundOperationsFromFile("test_model_loose/raw_output.txt")[0]
compoundOps

[COMPOUND_REPLACE 0:1 -> ['public']]

In [ ]:
compoundOps == getCondensedLoose(operations)

True

Nice! But what was the correct answer, and how far off were we?

In [ ]:
actualFixedMethod = readAbstractMethodsFromFile("../data/abstract_methods/small/test_fixed.txt")[0]
actualFixedMethod

private TYPE_1 getType ( TYPE_2 VAR_1 ) { TYPE_3 VAR_2 = new TYPE_3 ( STRING_1 ) ; return new TYPE_1 ( VAR_2 , VAR_2 , this , VAR_1 ) ; }

In [ ]:
modelDistance = outputMethod.getEditDistanceTo(actualFixedMethod)
modelDistance

5

In [ ]:
actualDistance = buggyMethod.getEditDistanceTo(actualFixedMethod)
actualDistance

4

Since `modelDistance` is higher than `actualDistance`, our outputted method is actually further away from the actual fixed method than the original buggy method is! Oof. But keep in mind that this is only demonstrating example usage and that the model was trained with a laughable number of steps.